# SWIFTCON 2024 - DMO Example

This notebook takes you through an example of running a dark matter only simulation and creating halo catalogues.

Created by Dr. Victor J. Forouhar Moreno PhD (forouhar@strw.leidenuniv.nl) & Ron (mcgibbon@strw.leidenuniv.nl)

## Dependencies

Running all these codes requires a number of other libraries, although there is considerable overlap.

monofonIC
- FFTW
- GSL
- HDF5

SWIFT
- HDF5 
- MPI
- FFTW
- METIS
- GSL

HBTplus
- CMake
- HDF5
- MPI

SOAP
- mpi4py
- h5py built with parallel HDF5
- Standard python modules (see requirements.txt)

#### Setup on cosma

This cell sets up dependencies for those running on cosma

In [1]:
module purge
module load python/3.12.4 gnu_comp/14.1.0 openmpi/5.0.3 parallel_hdf5/1.12.3
module load fftw gsl parmetis
venv_name="$(echo $HOME | sed 's/home/apps/')/swift_workshop_env"
python="$venv_name/bin/python3"

## monofonIC - Generate Initial Conditions

### Clone repo and compile

In [2]:
git clone https://github.com/wullm/monofonic.git
mkdir monofonic/build
cd monofonic/build
cmake ..
make -j 4
cd ../..

fatal: destination path 'monofonic' already exists and is not an empty directory.
mkdir: cannot create directory ‘monofonic/build’: File exists
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
CMake Warning (dev) at CMakeLists.txt:79 (find_package):
  Policy CMP0074 is not set: find_package uses <PackageName>_ROOT variables.
  Run "cmake --help-policy CMP0074" for policy details.  Use the cmake_policy
  command to set the policy and suppress this warning.

  Environment variable MPI_ROOT is set to:

    /cosma/local/openmpi/gnu_14.1.0/5.0.3

  For compatibility, CMake is ignoring the variable.
This warning is for project developers.  Use -Wno-dev to suppress it.

-- Found MPI: TRUE (found version "3.1")  
-- MPI_CXX_COMPILER = /cosma/local/openmpi/gnu_14.1.0/5.0.3/bin/mpicxx
-- MPI_CXX_INCLUDE_PATH = /cosma/local/parmetis/gnu_14.1.0_ompi_5.0.3/4.0.3/include;/cosma/local/gsl/2.8/include;

#### Parameter file

In [3]:
cat > monofonic.conf << EOF
[setup]

GridRes         = 64       # number of grid cells per linear dimension for calculations 
                           #   = particles for sc initial load
BoxLength       = 70       # length of the box in Mpc/h
zstart          = 49.0     # starting redshift

LPTorder        = 2        # order of the LPT to be used (1,2 or 3)

DoBaryons       = no       # also do baryon ICs?
DoBaryonVrel    = no       # if doing baryons, incl. also relative velocity to linear order?

DoFixing        = no       # do mode fixing à la Angulo&Pontzen (https://arxiv.org/abs/1603.05253)
DoInversion     = no       # invert phases (for paired simulations)

ParticleLoad    = sc       # particle load, can be 'sc' (1x), 'bcc' (2x) or 'fcc' (4x) 
                           # (increases number of particles by given factor!), 
                           # or 'glass' or 'masked'

DoDensityVelocityCorr = yes # 1st order velocity correction: account for the initial difference between theta_cdm and delta_cdm on large scales

WithNeutrinos =  yes       # master switch for enabling the following neutrino corrections

# Setting WithNeutrinos = yes enables each of the following (off by default). Uncomment for specific configurations.
# ExcludeNeutrinos = no      # 0th order correction: set cdm particle masses proportional to Omega_cdm and not Omega_cdm + Omega_nu_massive
# CDMBaryonMatterOnly = no   # 1st order correction: exclude neutrinos from the delta_matter and theta_matter transfer functions, i.e. dm = fb * db + fc * dc.
# DoNeutrinoPhi2Corr = no    # 2nd order correction: set the coefficient of the second order potential equal to the asymptotic value
# DoNeutrinoPhi3Corr = no    # 3rd order correction: set the coefficient of the third order potential equal to the asymptotic value

#########################################################################################
# Settings for neutrino particle initial conditions
# Note that these settings are distinct from the neutrino settings above, which
# only account for the presence of neutrinos in the dark matter & baryon ICs.
#
# Only the first neutrino species is generated, but degeneracy is taken into account.
# For example, for a standard 0.3 eV model, define in the cosmology section below:
#
# m_nu1 = 0.10
# m_nu2 = 0.0 # not used
# m_nu3 = 0.0 # not used
# deg_nu1 = 3.0
# deg_nu2 = 0.0 # not used
# deg_nu3 = 0.0 # not used

# DoNeutrinoParticles = yes  # generate neutrino particles with FastDF?
# NeutrinoCubeRootNum = 32   # N, where the number of neutrino particles is N^3
# NeutrinoGridRes = 32       # force mesh resolution for integrating neutrinos (should be <= GridRes, typically much smaller)
# NeutrinoStepSize = 0.05    # step size (delta log a) for integrating neutrinos
# NeutrinoInterpOrder = 1    # interpolation order (1 = NGP, 2 = CIC): NGP is much faster but less accurate

#########################################################################################
[cosmology]
## transfer = ... specifies the Einstein-Boltzmann plugin module

# ParameterSet    = Planck2018EE+BAO+SN  # specify a pre-defined parameter set, or set to 'none' and set manually below

## cosmological parameters, to set, choose ParameterSet = none,
## default values (those not specified) are set to the values
## from 'Planck2018EE+BAO+SN', we currently assume flatness
ParameterSet = none
Omega_m         = 0.3158
Omega_b         = 0.0494
Omega_L         = 0.6842 # not used (inferred from Omega_tot = 1)
H0              = 67.321
# Omega_m         = 0.3
# Omega_b         = 0.0
# Omega_L         = 0.6842 # not used (inferred from Omega_tot = 1)
# H0              = 70
n_s             = 0.9661
sigma_8         = 0.8102
A_s             = 2.148752e-09  # can use A_s instead of sigma_8 when using CLASS or zwindstroom
Tcmb            = 2.7255
k_p             = 0.05
N_ur            = 2.046
m_nu1           = 0.06
m_nu2           = 0.0
m_nu3           = 0.0
deg_nu1         = 1.0
deg_nu2         = 1.0
deg_nu3         = 1.0
w_0             = -1.0
w_a             = 0.0

ZeroRadiation   = false  # For Back-scaling only: set to true if your simulation code 
                         # cannot deal with Omega_r!=0 in its background FLRW model

## Use below for anisotropic large scale tidal field ICs up to 2LPT
## see Stuecker+2020 (https://arxiv.org/abs/2003.06427)
# LSS_aniso_lx    = +0.1
# LSS_aniso_ly    = +0.1
# LSS_aniso_lz    = -0.2

### MODULES: ###

##> Eisenstein & Hu (1997) fitting formulae 
## this is fast, but not too accurate. Also baryons trace CDM here. 
## see https://arxiv.org/abs/astro-ph/9709112

# transfer        = eisenstein   

##> CAMB transfer function file module
## This should be transfer function output with CAMB (https://camb.info)
## at the *target* redshift

# transfer        = file_CAMB    # CAMB file to be specified as 'transfer_file = ...'
# transfer_file   = wmap5_transfer_out_z0.dat

##> CLASS module, which links to the actual CLASS C-code.
## note that CLASS needs to be cloned as a git submodule and enabled in CMake file

# transfer        = CLASS
# ztarget         = 2.5             # target redshift for CLASS module, output at ztarget will be back-scaled to zstart

##> For neutrino models, we provide the zwindstroom module, which links both to
## the CLASS and zwindstroom C-codes. The former is used to provide the initial
## transfer functions and the latter for scale-dependent growth factors from a
## calculation of a Newtonian 3-fluid model with baryons + cdm + neutrinos.
## Unlike the CLASS plugin, this also outputs (nearly) exact N-body gauge
## transfer functions.
## It will also account for neutrinos in the Hubble rate, which enters
## into the conversion factor between displacements and velocities.
## Note that both CLASS and zwindstroom need to be cloned as git submodules
## and enabled in the CMake file.

transfer        = zwindstroom
ztarget         = 0.0          # target redshift, output at ztarget will be back-scaled to zstart


#########################################################################################
[random]
##> The PANPHASIA generator uses a plugin based on original code by A. Jenkins
## Before using this module, please make sure you read and agree to the distinct license
## requirements by registering on the website http://icc.dur.ac.uk/Panphasia.php

generator      = PANPHASIA
descriptor     = [Panph1,L10,(800,224,576),S9,CH1564365824,MXXL]
PanphasiaMinRootResolution = 64 # requires the white noise reallisation to be made at least at that resolution (default is 512)


#########################################################################################
[execution]
# Specify the number of threads / task
NumThreads      = 8


#########################################################################################
[output]
##> SWIFT compatible HDF5 format.
##> No h-factors for position and masses and no sqrt(a)-factor for the velocities.
##> IDs are stored using 64-bits unless UseLongids is set to false.
format          = SWIFT
filename        = small_cosmo_volume.hdf5
UseLongids      = false

# format          = gadget_hdf5
# filename        = small_cosmo_volume.hdf5

EOF

#### Run the code

In [4]:
./monofonic/build/monofonIC monofonic.conf

 
 The unigrid version of MUSIC-2         .8888b                   dP  a88888b. 
                                        88   "                   88 d8'   `88 
  88d8b.d8b. .d8888b. 88d888b. .d8888b. 88aaa  .d8888b. 88d888b. 88 88        
  88'`88'`88 88'  `88 88'  `88 88'  `88 88     88'  `88 88'  `88 88 88        
  88  88  88 88.  .88 88    88 88.  .88 88     88.  .88 88    88 88 Y8.   .88 
  dP  dP  dP `88888P' dP    dP `88888P' dP     `88888P' dP    dP dP  Y88888P' 

 Version: git rev.: 387ac9f, tag: N/A, branch: master
 This RelWithDebInfo build was compiled at 10:11:12 on Sep 18 2024
 Compiled with GNU C++ version 14.1.0
 -------------------------------------------------------------------------------
 Compile time options : 
                        Precision : double
                     Convolutions : Orszag3/2
                              PLT : PLT corr. off
 -------------------------------------------------------------------------------
 CPU vendor string                : AM

# SWIFT - Running simulation

[SWIFT](https://swift.strw.leidenuniv.nl/docs/index.html) is an open-source cosmological and astrophysical numerical solver designed to run efficiently on modern hardware. A comprehensive and extensive set of models for galaxy formation as well as planetary physics are provided alongside a large series of examples.

#### Clone repository

In [5]:
git clone https://gitlab.cosma.dur.ac.uk/swift/swiftsim.git
cd swiftsim

fatal: destination path 'swiftsim' already exists and is not an empty directory.


#### Configure and compile

See the [notes here](https://swift.strw.leidenuniv.nl/docs/GettingStarted/compiling_code.html)

Since HBTplus requires FOF information, we need to pass the `--enable-fof` flag

In [6]:
./autogen.sh
./configure --enable-fof

checking for a BSD-compatible install... /usr/bin/install -c
checking whether build environment is sane... yes
checking for a thread-safe mkdir -p... /usr/bin/mkdir -p
checking for gawk... gawk
checking whether make sets $(MAKE)... yes
checking whether make supports nested variables... yes
checking whether to enable debugging... no
checking whether make supports the include directive... yes (GNU style)
checking for gcc... /cosma/local/gcc/14.1.0/bin/gcc
checking whether the C compiler works... yes
checking for C compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether we are using the GNU C compiler... yes
checking whether /cosma/local/gcc/14.1.0/bin/gcc accepts -g... yes
checking for /cosma/local/gcc/14.1.0/bin/gcc option to accept ISO C89... none needed
checking whether /cosma/local/gcc/14.1.0/bin/gcc understands -c and -o together... yes
checking dependenc

In [7]:
make -j 4
cd ..

make  all-recursive
make[1]: Entering directory '/cosma/home/dp004/dc-mcgi1/SWIFTCON2024/swiftsim'
Making all in src
make[2]: Entering directory '/cosma/home/dp004/dc-mcgi1/SWIFTCON2024/swiftsim/src'
if test "X/usr/bin/git" != "X"; then \
    GIT_REVISION=`/usr/bin/git describe --abbrev=8  --always --tags --dirty 2>/dev/null`; \
    GIT_BRANCH=`/usr/bin/git branch 2>/dev/null | sed -n 's/^\* \(.*\)/\1/p' 2>/dev/null`; \
            GIT_DATE=`/usr/bin/git log -1 --format=%ci 2>/dev/null`; \
    sed -e "s,@PACKAGE_VERSION\@,1.0.0," \
        -e "s,@GIT_REVISION\@,${GIT_REVISION}," \
        -e "s|@GIT_BRANCH\@|${GIT_BRANCH}|" \
        -e "s|@GIT_DATE\@|${GIT_DATE}|" \
        -e "s|@SWIFT_CFLAGS\@|-flto -O3 -fomit-frame-pointer -malign-double -fstrict-aliasing -ffast-math -funroll-loops -march=znver2 -mavx2 -pthread -Wall -Wextra -Wno-unused-parameter -Wshadow -Werror -Wno-alloc-size-larger-than -Wstrict-prototypes|" version_string.h.in > version_string.h; \
else \
    if test ! -f vers

#### Set parameters

Point the code to the initial conditions we generated

Make sure to enable running the FOF before writing a snapshot

Set the number of output snapshots

In [8]:
cat > small_cosmo_volume_dm.yml << EOF
# Define the system of units to use internally. 
InternalUnitSystem:
  UnitMass_in_cgs:     1.98841e43    # 10^10 M_sun
  UnitLength_in_cgs:   3.08567758e24 # 1 Mpc
  UnitVelocity_in_cgs: 1e5           # 1 km/s
  UnitCurrent_in_cgs:  1             # Amperes
  UnitTemp_in_cgs:     1             # Kelvin


Cosmology:

# WMAP9 cosmology

  Omega_cdm: 0.3145
  Omega_b:          0.0
  Omega_lambda:          0.6855 # not used (inferred from Omega_tot = 1)
  h:              0.673


  # Omega_cdm:      0.3
  # Omega_lambda:   0.6842
  # Omega_b:        0.0494
  # h:              0.673
  a_begin:        0.02      	     # z_ini = 50.
  a_end:          1.0		         # z_end = 0.
  
  # Omega_cdm:      0.2305
  # Omega_lambda:   0.724
  # Omega_b:        0.0455
  # h:              0.703
  # a_begin:        0.0196

  # Omega_cdm:      0.3099
  # Omega_lambda:   0.6901
  # Omega_b:        0
  # h:              0.673
  # a_begin:        0.02      	     # z_ini = 50.
  # a_end:          1.0		         # z_end = 0.

# Parameters governing the time integration
TimeIntegration:
  dt_min:     1e-6 
  dt_max:     1e-2 

# Parameters for the self-gravity scheme
Gravity:
  eta:          0.025
  MAC:          adaptive
  theta_cr:     0.7
  epsilon_fmm:  0.001
  comoving_DM_softening:     0.0889     # 1/25th of the mean inter-particle separation: 88.9 kpc
  max_physical_DM_softening: 0.0889     # 1/25th of the mean inter-particle separation: 88.9 kpc
  mesh_side_length:       64

# Parameters governing the snapshots
Snapshots:
  basename:            snap
  delta_time:          1.15
  scale_factor_first:  0.1     # z = 9
  compression:         4
  invoke_fof:          1

# Parameters for running the friend-of-friends algorithm
FOF:  
  basename: fof_output        # Filename for the FOF outputs.
  min_group_size: 20          # The minimum no. of particles required for a group.
  linking_length_ratio: 0.2   # Linking length in units of the mean inter-particle separation
  seed_black_holes_enabled: 0
  linking_types: [0, 1, 0, 0, 0, 0, 0]         # Use DM as the primary FOF linking type
  attaching_types: [0, 0, 0, 0, 0, 0, 0]

# Parameters governing the conserved quantities statistics
Statistics:
  delta_time:          1.01
  scale_factor_first:  0.03

# Parameters governing the cell tree
Scheduler:
  max_top_level_cells: 8
  cell_split_size:     50
  
# Parameters related to the initial conditions
InitialConditions:
  file_name:                   small_cosmo_volume.hdf5
  periodic:                    1
  #cleanup_h_factors:           1    
  #cleanup_velocity_factors:    1
EOF

#### Run the code

In [9]:
./swiftsim/swift --cosmology --self-gravity --fof --threads=8 small_cosmo_volume_dm.yml

 Welcome to the cosmological hydrodynamical code
    ______       _________________
   / ___/ |     / /  _/ ___/_  __/
   \__ \| | /| / // // /_   / /   
  ___/ /| |/ |/ // // __/  / /    
 /____/ |__/|__/___/_/    /_/     
 SPH With Inter-dependent Fine-grained Tasking

 Version : 1.0.0
 Revision: v1.0.0-331-g5b139c59, Branch: master, Date: 2024-09-16 10:25:05 +0000
 Webpage : www.swiftsim.com

 Config. options: '--enable-fof'

 Compiler: GCC, Version: 14.1.0
 CFLAGS  : '-flto -O3 -fomit-frame-pointer -malign-double -fstrict-aliasing -ffast-math -funroll-loops -march=znver2 -mavx2 -pthread -Wall -Wextra -Wno-unused-parameter -Wshadow -Werror -Wno-alloc-size-larger-than -Wstrict-prototypes'

 HDF5 library version     : 1.12.3
 FFTW library version     : 3.x (details not available)
 GSL library version      : 2.8

[00000.0] main: CPU frequency used for tick conversion: 2894530214 Hz
[00000.0] main: Running on: login8a.pri.cosma.local
[00000.0] main: sizeof(part)        is  160 bytes.
[0

#### Output of run

- `snap_xxxx.hdf5` - Particles and their properties
- `fof_output_xxxx.hdf5` - FOF halo catalogue (FOF IDs stored in snapshots)
- `statistics.txt` - 
- `timesteps.txt` - 

In [10]:
ls

backup_monofonic.conf      HBT_config.txt              snap_0008.hdf5
cosma_env.sh               HBT_output                  snap_0009.hdf5
dependency_graph_0.csv     HBTplus                     snap_0010.hdf5
example.conf               input_class_parameters.ini  snap_0011.hdf5
fof_output_0000.hdf5       input_hubble.txt            snap_0012.hdf5
fof_output_0001.hdf5       input_powerspec.txt         snap_0013.hdf5
fof_output_0002.hdf5       input_transfer.txt          snap_0014.hdf5
fof_output_0003.hdf5       LICENSE                     snap_0015.hdf5
fof_output_0004.hdf5       membership_0018.hdf5        snap_0016.hdf5
fof_output_0005.hdf5       monofonic                   snap_0017.hdf5
fof_output_0006.hdf5       monofonic.conf              snap_0018.hdf5
fof_output_0007.hdf5       output_k_space_alt.0        snap.xmf
fof_output_0008.hdf5       README.md                   SOAP
fof_output_0009.hdf5       requirements.txt            SOAP_config.yml
fof_output_0010.hdf5       restart 

# HBTplus - Halo finding

Some description of HBT



#### Clone

In [11]:
git clone https://github.com/SWIFTSIM/HBTplus.git
cd HBTplus

fatal: destination path 'HBTplus' already exists and is not an empty directory.


#### Configure and compile

Use `ccmake` to see what options are available

In [12]:
cmake -B$PWD/build -D HBT_USE_OPENMP=ON -D HBT_DM_ONLY=ON -D HBT_UNSIGNED_LONG_ID_OUTPUT=OFF

-- Configuring done (1.8s)
-- Generating done (0.7s)
-- Build files have been written to: /cosma/home/dp004/dc-mcgi1/SWIFTCON2024/HBTplus/build


In [13]:
cd build
make -j 4
cd ../..

[  3%] Built target testfuncs
[ 55%] Built target hbtfuncs
[ 70%] Built target test_argsort
[ 70%] Built target test_sort_by_hash
[ 70%] Built target test_build_pos_vel
[ 70%] Built target HBT
[ 83%] Built target test_exchange_counts
[ 85%] Built target test_pairwise_alltoallv
[ 85%] Built target test_mpi
[ 85%] Built target test_locate_ids_unique
[ 98%] Built target test_mergertree
[100%] Built target test_locate_ids_random
[100%] Built target test_myalltoall
[100%] Built target match_snapshot_ids


#### Set parameters

In [14]:
cat > HBT_config.txt << EOF
# Configuration file used to run on small DMO simulation 

# Compulsary Params
SnapshotPath ./                      # Location of snapshots
SnapshotFileBase snap                # Basename of snapshot files
HaloPath ./                          # Location of FOF ID files (normal snapshot for SWIFT)
SubhaloPath HBT_output               # Output directory
# For the swiftsim reader these will be read in from snapshots automatically
BoxSize -1
SofteningHalo -1
MaxPhysicalSofteningHalo -1

# Reader
SnapshotFormat swiftsim
GroupFileFormat swiftsim_particle_index
MinSnapshotIndex 0                  # First snapshot to use
MaxSnapshotIndex 18                 # Final snapshot to use
MaxConcurrentIO 8                   # Number of cores for IO
MinNumPartOfSub 20                  # Minimum number of particles in a subhalo

# Units
MassInMsunh 6.81e9                  # Removes h factors from the final output 
LengthInMpch 0.681                  # Removes h factors from the final output
VelInKmS 1
EOF

##### Run

Run with a single rank as we will use 8 threads

In [15]:
export OMP_NUM_THREADS=8
mpirun -np 1 ./HBTplus/build/HBT HBT_config.txt

HBT compiled using git branch: master and commit: c31532a40dad661e843abcecadef03144b0cc057 (clean)
Reading configuration file HBT_config.txt
Running ./HBTplus/build/HBT from snapshot 0 to 18 using configuration file HBT_config.txt
./HBTplus/build/HBT run using 1 mpi tasks, each with 8 threads
Configured with the following data type sizes (bytes):
  Real quantities    : 4
  Integer quantities : 8
  Particle velocities: 4
  Particle masses    : 4
  Size of Particle_t : 48
Skipping empty snapshot -1

262144 particles loaded at Snapshot 0(0)
0 groups loaded at snapshot 0(0)
Unbinding...
saving 0 subhalos to HBT_output/000
SnapshotIndex 0 done. It took 0.316341 seconds.

262144 particles loaded at Snapshot 1(1)
0 groups loaded at snapshot 1(1)
Unbinding...
saving 0 subhalos to HBT_output/001
SnapshotIndex 1 done. It took 0.301654 seconds.

262144 particles loaded at Snapshot 2(2)
0 groups loaded at snapshot 2(2)
Unbinding...
saving 0 subhalos to HBT_output/002
SnapshotIndex 2 done. It took 

#### Output of run

For each snapshot each HBT rank outputs two files.
- The `SubSnap_xxx.y.hdf5` files contain information about the bound subhalos identified by HBTplus, including their particles
- The `SrcSnap_xxx.y.hdf5` files contain a list of the particles associated with each subhalo, but which are not bound at the current snapshot. These files are only used if HBT if restarted, and so can be deleted once halo finding has been completed

In [16]:
echo "Contents of HBT_output:"
ls HBT_output
echo "Output for snapshot 18:"
ls HBT_output/018

Contents of HBT_output:
000  002  004  006  008  010  012  014  016  018             timing.log
001  003  005  007  009  011  013  015  017  Parameters.log
Output for snapshot 18:
SrcSnap_018.0.hdf5  SubSnap_018.0.hdf5


# SOAP - Calculating halo properties

Dependencies
- mpi4py
- h5py built with parallel HDF5
- Standard python modules (see requirements.txt)

#### Clone repo

In [17]:
git clone https://github.com/SWIFTSIM/SOAP.git

fatal: destination path 'SOAP' already exists and is not an empty directory.


: 128

In [ ]:
# TODO MERGE and delete
cd SOAP
git checkout document_param_file
cd ..

#### Parameter file

Set input and output directories



In [18]:
cat > SOAP_config.yml << EOF
#### Parameter file

# Values in this section are substituted into the other sections
# The simulation name (box size and resolution) and snapshot will be appended
# to these to get the full name of the input/output files/directories
Parameters:
  sim_dir: $(pwd)
  output_dir: $(pwd)
  scratch_dir: $(pwd)

# Location of the Swift snapshots:
Snapshots:
  # Use {snap_nr:04d} for the snapshot number and {file_nr} for the file number.
  filename: "{sim_dir}/snap_{snap_nr:04d}.hdf5"

# Which halo finder we're using, and base name for halo finder output files
HaloFinder:
  type: HBTplus
  filename: "{sim_dir}/HBT_output/{snap_nr:03d}/SubSnap_{snap_nr:03d}"
  fof_filename: "{sim_dir}/fof_output_{snap_nr:04d}.hdf5"

GroupMembership:
  # Where to write the group membership files
  filename: "{output_dir}/SOAP_uncompressed/membership_{snap_nr:04d}.hdf5"

HaloProperties:
  # Where to write the halo properties file
  filename: "{output_dir}/SOAP_uncompressed/halo_properties_{snap_nr:04d}.hdf5"
  # Where to write temporary chunk output
  chunk_dir: "{scratch_dir}/SOAP-tmp/{halo_finder}/"

ApertureProperties:
  properties:
    {}
  variations:
    {}
ProjectedApertureProperties:
  properties:
    {}
  variations:
    {}
SOProperties:
  properties:
    CentreOfMass: true
    CentreOfMassVelocity: true
    Concentration: true
    MassFractionSatellites: true
    MassFractionExternal: true
    NumberOfDarkMatterParticles: true
    SORadius: true
    SpinParameter: true
    TotalMass: true
  variations:
    200_crit:
      type: crit
      value: 200.0
    200_mean:
      type: mean
      value: 200.0
    500_crit:
      type: crit
      value: 500.0
SubhaloProperties:
  properties:
    CentreOfMass: true
    CentreOfMassVelocity: true
    NumberOfDarkMatterParticles: true
    MaximumCircularVelocity: true
    MaximumCircularVelocityUnsoftened: true
    MaximumCircularVelocityRadiusUnsoftened: true
    SpinParameter: true
    TotalMass: true
  variations:
    Bound:
      bound_only: true
filters:
  general:
    limit: 100
    properties:
      - BoundSubhalo/NumberOfGasParticles
      - BoundSubhalo/NumberOfDarkMatterParticles
      - BoundSubhalo/NumberOfStarParticles
      - BoundSubhalo/NumberOfBlackHoleParticles
    combine_properties: sum
  baryon:
    limit: 100
    properties:
      - BoundSubhalo/NumberOfGasParticles
      - BoundSubhalo/NumberOfStarParticles
    combine_properties: sum
  dm:
    limit: 100
    properties:
      - BoundSubhalo/NumberOfDarkMatterParticles
  gas:
    limit: 100
    properties:
      - BoundSubhalo/NumberOfGasParticles
  star:
    limit: 100
    properties:
      - BoundSubhalo/NumberOfStarParticles
calculations:
  recalculate_xrays: false
  calculate_missing_properties: false
  min_read_radius_cmpc: 5
  recently_heated_gas_filter:
    delta_time_myr: 15
    use_AGN_delta_T: true
EOF

In [20]:
mpirun -np 8 $python -u SOAP/group_membership.py \
    --sim-name=DM_test \
    --snap-nr=18 \
    SOAP_config.yml

lfs setstripe: setstripe error for '/cosma/home/dp004/dc-mcgi1/SWIFTCON2024/SOAP_uncompressed': Inappropriate ioctl for device
Number of group particle IDs per rank min=0, max=82411
Calculating group membership for type PartType1
  Number of snapshot particle IDs per rank min=32768, max=32768
  Matching SWIFT particle IDs to bound IDs
  Assigning bound group membership to SWIFT particles
  Assigning rank by binding energy to SWIFT particles
  Writing out group membership of SWIFT particles
Done.


In [21]:
mpirun -np 8 $python -u SOAP/compute_halo_properties.py \
    --sim-name=DM_test \
    --snap-nr=18 \
    --chunks=1 \
    --dmo \
    SOAP_config.yml

Starting halo properties calculation on 8 MPI ranks
Can process 1 chunks in parallel using 8 ranks per chunk
Number of MPI ranks per node reading snapshots: 32
Halo format is HBTplus
Halo basename is /cosma/home/dp004/dc-mcgi1/SWIFTCON2024/HBT_output/{snap_nr:03d}/SubSnap_{snap_nr:03d}
Output file is /cosma/home/dp004/dc-mcgi1/SWIFTCON2024/SOAP_uncompressed/halo_properties_{snap_nr:04d}.hdf5
Snapshot number is 18
Could not find lightcone observer position in snapshot file. Defaulting to centre of box.
Halo property calculations enabled:
  bound_subhalo_properties
  SO_200_crit
  SO_200_mean
  SO_500_crit
for central and satellite halos
Properties not present in the parameter file will not be calculated
Run in DMO mode
Category filters :
  general   100
  dm        100
  gas       100
  star      100
  baryon    100
lfs setstripe: setstripe error for '/cosma/home/dp004/dc-mcgi1/SWIFTCON2024/SOAP_uncompressed': Inappropriate ioctl for device
Using Peano domain decomposition with bits=10


#### SOAP output

#### Compression

The files output from SOAP can be heavily compressed. For the membership files this is because most particles are not in a halo, and there are many repeated indices for the ones that are. For the halo properties we do not compute certain properties depending on the size of the input halo, so many values are zero.

The membership files have no lossy compression filters, and can be compressed with h5repack

In [22]:
h5repack -i SOAP_uncompressed/membership_0018.hdf5 -o membership_0018.hdf5 -f GZIP=4

The halo catalogues have the same lossy compression filters as are available in SWIFT (some of which are custom), and so must be compressed using the following script.

In [23]:
python SOAP/compression/compress_fast_metadata.py \
    SOAP_uncompressed/halo_properties_0018.hdf5 \
    halo_properties_0018.hdf5 \
    SOAP_uncompressed/tmp

Copying over groups to halo_properties_0018.hdf5 and listing datasets...
File structure copy took 58.96 ms.
Copying over datasets to temporary files in SOAP_uncompressed/tmp/halo_properties_0018_temp using 1 processes...
Temporary file writing took 854.09 ms.                                                                        
Copying datasets into halo_properties_0018.hdf5 and cleaning up temporary files...
Temporary file copy took 113.07 ms.                                                                           
416.2KB -> 124.6KB (29.95%)
Total writing time: 1026.21 ms.


#### Generate documentation

Documentation can be generated by running the `property_table.py`. You must pass the parameter file (to determine which halo types and properties are included in the documentation) and a SWIFT snapshot (to extract the units).

In [ ]:
cd SOAP
$python property_table.py ../SOAP_config.yml ../snap_0018.hdf5
cd documentation
# pdflatex SOAP.tex
cd ../..